In [ ]:
%matplotlib widget
import os
import math
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import exponnorm
from scipy.stats import linregress
from scipy.special import erfc
import scipy.constants as const

from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.signal import find_peaks
from lmfit import Model

from SiPMStudio.analysis.dark import current_waveforms
from SiPMStudio.analysis.dark import integrate_current

from iminuit import Minuit
from iminuit.cost import ExtendedUnbinnedNLL
from iminuit.cost import UnbinnedNLL
from iminuit.cost import LeastSquares

from functools import partial
from tqdm.notebook import tqdm

In [ ]:
def zero_peak(x, A0, B0, x0, sigma_0, rate): 
    return A0*norm.pdf(x, x0, sigma_0) + B0*exponnorm.pdf(x, rate, x0, sigma_0)

def photon_peak(x, k, A0, B1, x0, sigma_0, sigma_1, gain, rate): 
    mean = x0 + k*gain 
    sigma = np.sqrt(sigma_0**2 + k*sigma_1**2) 
    return A0*(norm.pdf(x, mean, sigma) + B1*exponnorm.pdf(x, rate, mean, sigma))

def amplitude(x, tau, b): 
    return b*np.exp(-x/tau)

def amplitude_v2(k, lam):
    return poisson.pmf(k, lam)

def spectrum_pdf(x, A0, B0, B1, tau, b, x0, x1, sigma_0, sigma_1, gain, rate_0, rate_1, num_peaks=5): 
    pdf = zero_peak(x, A0, B0, x0, sigma_0, rate_0) 
    norm = A0 + B0
    for k in range(1, int(num_peaks+1)): 
        amp = amplitude(x1+k*gain, tau, b)
        #amp = b*amplitude_v2(k-1, tau)
        pdf += photon_peak(x, k, amp, B1, x1, sigma_0, sigma_1, gain, rate_1) 
        norm += amp * (1 + B1) 
    pdf = pdf / norm 
    return pdf

def height_pdf(x, B1, tau, b, x0, sigma_0, sigma_1, gain, rate_1, num_peaks=5): 
    norm = 0
    pdf = np.array([0]*len(x), dtype=float)
    for k in range(1, int(num_peaks+1)): 
        amp = amplitude(x0+k*gain, tau, b)
        pdf += photon_peak(x, k, amp, B1, x0, sigma_0, sigma_1, gain, rate_1) 
        norm += amp * (1 + B1) 
    pdf = pdf / norm 
    return pdf

In [ ]:
def gaussian(x, A, mu, sigma):
    return A*np.exp(-(x - mu)**2/(2*sigma**2))

def exp_func(x, A, tau):
    return A*np.exp(-x/tau)

def gaussian_fit(x, y):
    params, cov = np.polyfit(x, np.log(y), deg=2, cov=True)
    errors = np.sqrt(np.diag(cov))
    mu = -params[1]/(2*params[0])
    sigma = np.sqrt(-1/(2*params[0]))
    A = np.exp((mu**2/(2*sigma**2)) + params[2])

    A_err = np.sqrt((A**2/(16*params[0]**4))*errors[0]**2 + (4*A**2*errors[1]**2) + A**2*errors[2]**2)
    mu_err = sigma*np.sqrt(params[1]**2*errors[0]**2 + errors[1]**2)
    sigma_err = (-2*params[0])**(-3/2)*errors[0]
    
    A_param = [A, A-A_err, A+A_err]
    mu_param = [mu, mu-mu_err, mu + mu_err]
    sigma_param = [sigma, sigma-sigma_err, sigma+sigma_err]
    
    return A_param, mu_param, sigma_param

def guess_params(x, y, peaks, props, show=False):
    photon_peaks = peaks[1:]
    gain_average = x[photon_peaks[1]] - x[photon_peaks[0]]
    gain_std = gain_average/2
    gain = [gain_average, gain_average-gain_std, gain_average+gain_std]
    x0 = 0
    sigma_0 = 1
    sigma_1 = 1
    A0 = props["peak_heights"][0]
    mus = []
    amps = []
    rate = 0
    limits = []
    for i, peak in enumerate(peaks[:4]):
        x_fit = x[(x > x[peak] - (gain_average/2)) & (x < x[peak] + (gain_average/2))]
        y_fit = y[(x > x[peak] - (gain_average/2)) & (x < x[peak] + (gain_average/2))]
        
        x_fit = x_fit[y_fit > y[peak]/4]
        y_fit = y_fit[y_fit > y[peak]/4]
        A, mu, sigma = gaussian_fit(x_fit, y_fit)
        mu[1] = mu[0] - gain_average/4
        mu[2] = mu[0] + gain_average/4
        sigma[1] = 1e-8
        sigma[2] = sigma[0] + sigma[0]
        A[1] = A[0] - A[0]
        A[2] = A[0] + A[0]
        
        if i == 0:
            A0 = A
            x0 = mu
            sigma_0 = sigma
        elif i == 1:
            sigma_1 = sigma
            mus.append(mu[0])
            amps.append(A[0])
        else:
            mus.append(mu[0])
            amps.append(A[0])
    params_info = linregress(mus, np.log(amps))
    slope = params_info[0]
    intercept = params_info[1]
    slope_err = params_info[-2]
    int_err = params_info[-1]
    tau = [-1/slope, 0, -1/(slope) + (1/slope**2)*slope_err]
    A_exp = [np.exp(intercept), 0, 2*np.exp(intercept)]
    return x0, A0, gain, sigma_0, sigma_1, A_exp, tau

def guess_params_height(x, y, peaks, props, show=False):
    gain_average = x[peaks[1]] - x[peaks[0]]
    gain_std = gain_average/2
    gain = [gain_average, gain_average-gain_std, gain_average+gain_std]
    x0 = 0
    sigma_0 = 1
    sigma_1 = 1
    A0 = 0
    mus = []
    amps = []
    rate = 0
    limits = []
    
    for i, peak in enumerate(peaks[[0, 1]]):
        x_fit = x[(x > x[peak] - (gain_average/2)) & (x < x[peak] + (gain_average/2))]
        y_fit = y[(x > x[peak] - (gain_average/2)) & (x < x[peak] + (gain_average/2))]
        
        x_fit = x_fit[y_fit > y[peak]/4]
        y_fit = y_fit[y_fit > y[peak]/4]
        x_fit = np.array(x_fit, dtype=np.float64)
        y_fit = np.array(y_fit, dtype=np.float64)
        
        A, mu, sigma = gaussian_fit(x_fit, y_fit)
        mu[1] = mu[0] - gain_average/4
        mu[2] = mu[0] + gain_average/4
        sigma[1] = 1e-8
        sigma[2] = sigma[0] + sigma[0]
        A[1] = A[0] - A[0]
        A[2] = A[0] + A[0]
        
        if i == 0:
            sigma_0 = sigma
            sigma_1 = sigma
            mus.append(mu[0])
            amps.append(A[0])
        else:
            mus.append(mu[0])
            amps.append(A[0])
    params_info = linregress(mus, np.log(amps))
    slope = params_info[0]
    intercept = params_info[1]
    slope_err = params_info[-2]
    int_err = params_info[-1]
    tau = [-1/slope, 0, -1/(slope) + (1/slope**2)*slope_err]
    A_exp = [np.exp(intercept), 0, 2*np.exp(intercept)]
    return x0, A0, gain, sigma_0, sigma_1, A_exp, tau

def append_to_file(file, dataname, data):
    with h5py.File(file, "a") as h5_file:
        h5_file.create_dataset(dataname, data=data)
        
def overwrite_to_file(file, dataname, data):
    with h5py.File(file, "r+") as h5_file:
        del h5_file[dataname]
        h5_file.create_dataset(dataname, data=data)

In [ ]:
t1_path = "/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_cold1/t1"
t2_path = "/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_cold1/t2"
t1_file = os.path.join(t1_path, "t1_ketek_cold1_28.h5")
t2_file = os.path.join(t2_path, "t2_ketek_cold1_28.h5")

In [ ]:
t1_sipm_file = h5py.File(t1_file, "r")

In [ ]:
t1_waveforms = t1_sipm_file["/raw/waveforms"][:]
t1_baselines = t1_sipm_file["/raw/baselines"][:]
t1_sipm_file.close()

In [ ]:
t1_flatforms = t1_waveforms - t1_baselines

In [ ]:
t_plot = np.arange(0, 2*t1_flatforms.shape[1], 2)
plt.figure()
plt.plot(t_plot, t1_flatforms[4])
plt.xlabel("Time (ns)")
plt.ylabel("ADC")
plt.grid(alpha=0.25)

In [ ]:
current_forms = current_waveforms(t1_flatforms)
charges = integrate_current(current_forms, 35, 200)*1e12

In [ ]:
plt.figure()
[n, bins, patches] = plt.hist(charges, bins=1000, edgecolor="none", alpha=0.50, density=True)
n_plot = np.append(n, 0)
bins_plot = np.append(bins, bins[-1]+(bins[1]- bins[0]))
plt.step(bins_plot[1:], n_plot, color=sns.color_palette()[0])
plt.xlabel("Charge (pC)")
plt.ylabel("Norm Counts")
plt.yscale("log")

In [ ]:
bin_centers = (bins[:-1] + bins[1:]) / 2
n_centers = n
bin_width = bins[1] - bins[0]
scale = np.sum(n_centers)*bin_width

In [ ]:
peaks, props = find_peaks(n, height=1e-1, distance=30)

In [ ]:
props["peak_heights"]

In [ ]:
plt.figure()
[n, bins, patches] = plt.hist(charges, bins=1000, edgecolor="none", density=True, alpha=0.75)
plt.scatter(bin_centers[peaks], props["peak_heights"], c="red")
plt.xlabel("Charge (pC)")
plt.ylabel("Norm Counts")
#plt.xlim(-0.1, 0.5)
plt.yscale("log")

In [ ]:
x0, A0, gain, sigma_0, sigma_1, A_exp, tau = guess_params(bin_centers, n_centers, peaks, props, show=False)
gain

In [ ]:
likelihood = UnbinnedNLL(charges, pdf=spectrum_pdf)

In [ ]:
likelihood_fit = Minuit(likelihood, A0=A0[0], B0=0, B1=0.15, tau=tau[0], b=2, 
                        sigma_0=sigma_0[0], sigma_1=sigma_1[0], x0=x0[0], x1=x0[0], 
                        gain=gain[0], rate_0=1, rate_1=10, num_peaks=7)

In [ ]:
s = 1.5
likelihood_fit.limits["A0"] = (A0[1], A0[2])
likelihood_fit.limits["B1"] = (1e-2, 1)
likelihood_fit.limits["tau"] = (s*tau[1], s*tau[2])
likelihood_fit.limits["b"] = (1e-2, 100)
likelihood_fit.limits["sigma_0"] = (sigma_0[1], sigma_0[2])
likelihood_fit.limits["sigma_1"] = (sigma_1[1], sigma_1[2])
likelihood_fit.limits["x0"] = (x0[1], x0[2])
likelihood_fit.limits["gain"] = (gain[1], gain[2])
likelihood_fit.limits["rate_0"] = (1e-2, 10)
likelihood_fit.limits["rate_1"] = (1, 1e3)
likelihood_fit.fixed["B0"] = True
likelihood_fit.fixed["num_peaks"] = True

In [ ]:
likelihood_fit.migrad()

In [ ]:
x_plot = np.linspace(0, max(charges), 1000)

In [ ]:
plt.figure()
plt.plot(x_plot, scale*spectrum_pdf(x_plot, *likelihood_fit.values), color="magenta")
[n, bins, patches] = plt.hist(charges, bins=1000, edgecolor="none", density=True, alpha=0.75)
#plt.plot(x_plot, spectrum_pdf(x_plot, A0=1e-3, B0=0.5, B1=0.15, tau=4e-2, b=2, 
#                        sigma_0=sigma_0[0], sigma_1=sigma_1[0], x0=x0[0], 
#                        gain=gain[0], rate_0=1, rate_1=10, num_peaks=5), color="orange")
plt.xlabel("Charge (pC)")
plt.ylabel("Norm Counts")
plt.ylim(1e-3, 1e2)
plt.yscale("log")

In [ ]:
likelihood_fit.values

In [ ]:
gain_magnitude = likelihood_fit.values["gain"]*1e-12 / const.e
gain_magnitude

In [ ]:
likelihood_fit.errors["gain"]*1e-12 / const.e

In [ ]:
t2_sipm_file = h5py.File(t2_file, "r")

In [ ]:
t2_waveforms = t2_sipm_file["/processed/waveforms"][:]
t2_sipm_file.close()

In [ ]:
plt.figure()
plt.plot(t2_waveforms[4])

In [ ]:
def wave_peaks(waveforms, height=500, distance=5):
    all_peaks = []
    all_heights = []
    for waveform in tqdm(waveforms, total=len(waveforms)):
        peak_loc = find_peaks(waveform, height=height, distance=distance)[0]
        if len(peak_loc) > 0:
            if peak_loc[0] < 60:
                all_peaks.append(peak_loc[0])
                all_heights.append(waveform[peak_loc][0])
    return np.asarray(all_peaks, dtype=object), np.asarray(all_heights, dtype=object)

def amplitudes(heights):
    amps = []
    for i, height in tqdm(enumerate(heights), total=len(heights)):
        amps += list(height)
    return np.array(amps)

def waveform_maxes(waveforms, start=35, stop=80):
    section = waveforms.T[start:stop]
    return np.amax(section, axis=0)

In [ ]:
all_peaks, all_heights = wave_peaks(t2_waveforms, 1800, distance=100)
#all_heights = waveform_maxes(t2_waveforms)
#all_heights = all_heights[all_heights > 382]

In [ ]:
plt.figure()
[n, bins, patches] = plt.hist(all_heights, bins=1000, edgecolor="none", density=True, alpha=0.75)
plt.xlabel("Amplitude (A. U.)")
plt.ylabel("Norm Counts")
#plt.xlim(-0.1, 0.5)
plt.yscale("log")

In [ ]:
bin_centers = (bins[:-1] + bins[1:]) / 2
n_centers = n
bin_centers = bin_centers
bin_width = bins[1] - bins[0]
scale = np.sum(n_centers)*bin_width

In [ ]:
peaks, props = find_peaks(n, height=1e-5, distance=20)

In [ ]:
props["peak_heights"]

In [ ]:
plt.figure()
[n, bins, patches] = plt.hist(all_heights, bins=1000, edgecolor="none", density=True, alpha=0.75)
plt.scatter(bin_centers[peaks], props["peak_heights"], c="red")
plt.xlabel("Charge (A. U.)")
plt.ylabel("Norm Counts")
#plt.xlim(-0.1, 0.5)
plt.yscale("log")

In [ ]:
x0, A0, gain, sigma_0, sigma_1, A_exp, tau = guess_params_height(bin_centers, n_centers, peaks, props, show=False)

In [ ]:
likelihood = UnbinnedNLL(all_heights, pdf=height_pdf)

In [ ]:
likelihood_fit = Minuit(likelihood, B1=0, tau=tau[0], b=A_exp[0], 
                        sigma_0=sigma_0[0], sigma_1=sigma_1[0], x0=x0, 
                        gain=gain[0], rate_1=1, num_peaks=7)

In [ ]:
likelihood_fit.limits["x0"] = (-1e3, 500)
likelihood_fit.limits["gain"] = (gain[1], gain[2])
likelihood_fit.limits["sigma_0"] = [sigma_0[1], sigma_0[2]]
likelihood_fit.limits["sigma_1"] = [sigma_1[1], sigma_1[2]]
likelihood_fit.limits["tau"] = [1e-6, 3e3]
likelihood_fit.limits["b"] = [0, 1e4]
likelihood_fit.fixed["B1"] = True
likelihood_fit.fixed["num_peaks"] = True
likelihood_fit.fixed["rate_1"] = True

In [ ]:
likelihood_fit.migrad()

In [ ]:
plt.figure()
num_range = 15000
x_plot = np.linspace(0, num_range, num_range)
[n, bins, patches] = plt.hist(all_heights, bins=1000, edgecolor="none", alpha=0.75, density=True)
plt.plot(x_plot, height_pdf(x_plot, *likelihood_fit.values), color="magenta")
plt.xlabel("Amplitudes (A. U.)")
plt.ylabel("Norm Counts")
plt.yscale("log")
plt.ylim(1e-6, 1e-1)

In [ ]:
likelihood_fit.values

In [ ]:
def waveform_calibration(waveforms, x0, gain):
    return (waveforms - x0) / gain

In [ ]:
#plt.close("all")

In [ ]:
norm_waveforms = waveform_calibration(t2_waveforms, likelihood_fit.values["x0"], likelihood_fit.values["gain"])
norm_heights = waveform_calibration(all_heights, likelihood_fit.values["x0"], likelihood_fit.values["gain"])

In [ ]:
plt.figure()
plt.plot(norm_waveforms[4])

In [ ]:
plt.figure()
[n, bins, patches] = plt.hist(norm_heights, bins=1000, edgecolor="none", density=True, alpha=0.75)
plt.xlabel("Amplitude (A. U.)")
plt.ylabel("Norm Counts")
#plt.xlim(-0.1, 0.5)
plt.yscale("log")

In [ ]:
append_to_file(t2_file, "gain", gain_magnitude)
append_to_file(t1_file, "gain", gain_magnitude)
overwrite_to_file(t2_file, "/processed/waveforms", norm_waveforms)

In [ ]:
#append_to_file(t2_file, "gain", 2340888.1583642513)